In [1]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import casadi as csd

In [2]:
class csdModel:
    pass

In [3]:
model = csdModel()
model.transcription = "shooting"
model.x = ['ca', 'cb', 'cc']
model.p = ['p1', 'p2']
model.d = ['m_ca', 'm_cb', 'm_cc']

In [4]:
def massbal_ode(t, x, p):
    r1 = p.k1 * x.ca
    r2 = p.k2 * x.cb
    xdot = [-r1, r1 - r2, r2]
    return xdot

In [5]:
model.ode = lambda t, x, p: massbal_ode

In [6]:
model.lsq = lambda t, y, p: [y.ca-y.m_ca, y.cb-y.m_cb, y.cc-y.m_cc]

In [7]:
nmeas = [1, 2, 3]

In [8]:
tfinal = 6
nplot = 75
tplot = np.linspace(0, tfinal, nplot)

In [9]:
tabledat = pd.read_csv('ABC_data.csv')

In [10]:
tmeas = tabledat.iloc[:,0].values
ymeas = tabledat.iloc[:,[1, 2, 3]].to_numpy()

In [11]:
tout = np.unique(np.concatenate([tmeas, tplot]))
model.tout = tout

In [12]:
meas_ind = np.where(np.in1d(tout, tmeas))[0]

In [13]:
ynoisy = np.zeros((len(tout), 3))
for i in range(3):
    ynoisy[:, i] = interp1d(tmeas, ymeas[:, i], kind = 'previous', fill_value = 'extrapolate')(tout)

In [14]:
model.lsq_ind = meas_ind

In [15]:
f = ['x', 'z', 'p', 'y']
for fi in f:
    if not hasattr(model, fi):
        setattr(model, fi, [])

In [16]:
nsets = 1

In [17]:
model.nlp_solver = 'ipopt'

In [18]:
class solverOptions:
    pass

In [19]:
model.nlp_solver_options = solverOptions()
model.nlp_solver_options.calc_lam_x = True
model.nlp_solver_options.calc_lam_p = True

In [20]:
t = csd.SX.sym('t')

In [ ]:
# start with empty NLP
w = []
lsq = []
g = []

In [ ]:
# w with internal variables eliminated
w_elim = []

In [ ]:
# x, y, z, d trajectories
x = []
z = []
d = []

In [ ]:
model.N = len(model.tout) - 1
model.np = len(model.p)

In [ ]:
lsq_ind = np.zeros(model.N + 1)

In [ ]:
lsq_ind[model.lsq_ind] = 1

In [ ]:
if hasattr(model, 'lsq_ind'):
    lsq_ind[model.lsq_ind] = 1
else:
    lsq_ind[:] = 1

In [ ]:
# sensitivity parameters
sens = []

In [ ]:
# free parameter
p = csd.MX.sym('p', model.np)

In [ ]:
w.append(p)
w_elim.append(p)

In [ ]:
t = model.tout
h = t[1:] - t[0:-1]

In [ ]:
model.nsets = 1
for i in range(model.nsets):
    if model.nsets > 1:
        s = '-' + str(i)
    else:
        s = ''

In [ ]:
# time points
tk = t[0]

In [ ]:
tk

In [ ]:
model.nx = len(model.x)
model.nz = len(model.z)

In [ ]:
xk = csd.MX.sym('x0'+s, model.nx)

In [ ]:
w.append(xk)
x.append(xk)
w_elim.append(xk)

In [ ]:
zk = csd.MX.sym('z0' + s, model.nz)

In [ ]:
w.append(zk)
x.append(zk)
w_elim.append(zk)

In [ ]:
sens.append(xk)